In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep, time
from random import randint

In [0]:
urlGenres = "http://www.songlyrics.com/musicgenres.php"
response = get(urlGenres)

In [4]:
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [0]:
genre_containers = html_soup.find_all('div', class_ = 'box listbox')[0]

In [0]:
genres = []
names = []
artists = []
lyrics = []

for container in genre_containers.find_all('td', class_ = 'td-item td-text-big'):
  genre = container.a.text
  genreORG = genre
  genre = genre.replace(' ', '-').replace('/', '-').replace('&', '-and-') .replace('\'', '').lower().replace('country', 'country-music')

  # sleep(randint(8, 15))

  urlByGenre = "http://www.songlyrics.com/" + genre + "-lyrics.php"
  response = get(urlByGenre)

  html_soup = BeautifulSoup(response.text, 'html.parser')
  songs_containers = html_soup.find_all('td', class_ = 'td-item td-last')

  for run in range(5):
    genres.append(genreORG)
    song = songs_containers[run].a.text
    names.append(song)
    song = song.replace(' ', '-').replace('/', '-').replace('&', '-and-').replace("other's", "others").replace('\'', '-').replace('.', '').replace('?', '').lower()
    song = song.replace('--', '-').replace('---', '-')
    artist = songs_containers[run].span.text
    artists.append(artist)
    artist = artist.replace(' ', '-').replace('/', '-').replace('&', '-and-').replace("other's", "others").replace('\'', '-').replace('.', '').replace('?', '').lower()
    artist = artist.replace('--', '-').replace('---', '-')

    # sleep(randint(8, 15))

    urlLyrics = "http://www.songlyrics.com/" + artist + "/" + song + "-lyrics/" 
    response = get(urlLyrics)

    # print(genre + " | " + song + " | " + artist)  
    # print(urlLyrics)

    html_soup = BeautifulSoup(response.text, 'html.parser')

    lyrics_containers = html_soup.find_all('p', class_ = 'songLyricsV14 iComment-text')
    lyric = lyrics_containers[0].text

    lyrics.append(lyric)

In [7]:
# pd.set_option('display.max_colwidth', -1)
songInfoLists = pd.DataFrame({'genre' : genres
                       , 'name' : names
                       , 'artist' : artists
                       , 'lyric' : lyrics})
songInfoLists.shape

(160, 4)

In [0]:
# songInfoLists[songInfoLists['genre'] == 'Acoustic']

In [9]:
songInfoLists = songInfoLists[songInfoLists['lyric'].str.len() > 300]

songInfoLists = songInfoLists.replace(r'\\n',' ', regex=True)
songInfoLists = songInfoLists.replace('\n',' ', regex=True)
songInfoLists = songInfoLists.replace(r'\\n',' ', regex=True)
songInfoLists = songInfoLists.replace(r'\r',' ', regex=True)

songInfoLists.reset_index(drop=True, inplace=True)
songInfoLists.shape

(140, 4)

In [10]:
songInfoLists['genre'].value_counts()

Pop                   5
Soundtrack            5
Latin                 5
Christian             5
Musical               5
Blues                 5
Comedy                5
Hip Hop/Rap           5
New Age               5
Electronic            5
Rock                  5
World                 5
Children's Music      5
Adult Contemporary    5
African               5
Vocal                 5
Dance                 5
Oldies                4
Ska                   4
Country               4
Jazz                  4
Reggae                4
R&B                   4
Acoustic              4
Classical             4
Folk                  4
Soul                  4
Alternative           3
Funk                  3
Instrumental          3
Avant-Garde           3
Holiday               3
Name: genre, dtype: int64

In [11]:
songInfoLists.head()

,genre,name,artist,lyric
0,Acoustic,Dream,Priscilla Ahn,Priscilla Ahn A Good Day Dream I was a little ...
1,Acoustic,Love Songs On The Radio,Mojave,Mojave Love Songs On The Radio Love Songs On T...
2,Acoustic,"Sitting, Waiting, Wishing",Johnson Jack,"Johnson Jack In Between Dreams Sitting, Waitin..."
3,Acoustic,You Got Growin Up To Do,Joshua Radin,I've been down this road before I walk out the...
4,Alternative,Blow Away,A Fine Frenzy,One of us is gonna be here and And one of us i...


In [0]:
fullDataPath = "/content/gdrive/My Drive/DS532_Text_Analytic/Homework/songData/";

In [0]:
# Delete all file in folder
import os, shutil
for filename in os.listdir(fullDataPath):
    file_path = os.path.join(fullDataPath, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [0]:
# Write lyric to text file
for songInfo in songInfoLists.itertuples():
  name = songInfo.name.lower();
  artist = songInfo.artist.lower();
  lyric = songInfo.lyric.lower();

  file = open(fullDataPath + name.replace('.', '') + " - " + artist.replace('.', '') + ".txt", "w")
  file.write(lyric) 
  file.close()